In [1]:
# from cProfile import label


# a = [1, 3, 4]
# b = [5, 4, 2]
# print(list(map(lambda (x, y): x+y, zip(a, b))))

In [2]:
from math import sqrt
import pdb
from typing import List

from openfoam.optimizer import T, last


FLOAT_MACHINE_EPSILON = 2.22e-16
FLOAT_EPSILON = sqrt(FLOAT_MACHINE_EPSILON)


class FloatGd(T):
    def __init__(self, inner: float, eta: float) -> None:
        super().__init__()
        self.inner = inner
        self.eta = eta

    def succ(self) -> "FloatGd":
        return FloatGd(self.inner + FLOAT_EPSILON, self.eta)

    def partial_diff(self, y: float, succ_y: float) -> float:
        pd = (succ_y - y)/FLOAT_EPSILON
        return pd
    
    def update(self, pds: List[float]) -> T:
        pd = last(pds)
        return FloatGd(self.inner - self.eta * pd, self.eta)

    def __repr__(self):
        return f"Float<inner: {self.inner}>"

In [3]:
from openfoam.optimizer import optimize

# def stop_if_small_y(_thetas, ys, _pdss) -> bool:
#     return last(ys) < 0.01

# def log_by_print(thetas, ys, pdss):
#     print("Interation: ", len(ys))
#     print("Theta: ", list(map(last, thetas)))
#     print("y: ", last(ys))
#     print("pds: ", list(map(last, pdss)), "\n")


# def square(ts: List[T]) -> float: 
#     # print("ts of square: ", ts)
#     return (ts[0].inner - 1) ** 2

# optimize(square, [[FloatGd(100., 0.1)]], [], [[]], stop_if_small_y, log_by_print)

In [4]:
from copy import deepcopy


class FloatAdam(T):
    def __init__(
            self, inner: float, 
            alpha: float, 
            beta1: float = 0.9, 
            beta2: float=0.99
            ) -> None:
        
        super().__init__()
        self.inner = inner
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.m1 = 0.
        self.m2 = 0.

    def succ(self) -> "FloatGd":
        new = deepcopy(self)
        new.inner = self.inner + FLOAT_EPSILON
        return new

    def partial_diff(self, y: float, succ_y: float) -> float:
        pd = (succ_y - y)/FLOAT_EPSILON
        return pd
    
    def update(self, pds: List[float]) -> T:
        pd = last(pds)
        m1_ = self.beta1 * self.m1 + (1-self.beta1) * pd
        m2_ = self.beta2 * self.m2 + (1-self.beta2) * (pd**2)
        inner_ = self.inner - self.alpha * (m1_ / sqrt(m2_ + FLOAT_EPSILON))

        new = deepcopy(self)
        new.m1 = m1_
        new.m2 = m2_
        new.inner = inner_

        return new

    def __repr__(self):
        return f"Float<inner: {self.inner}>"

In [11]:
from openfoam.graph import *
from openfoam.optimizer import *
from typing import List


def stop_if_small_y(_thetas, ys, _pdss) -> bool:
    return last(ys) < 0.01

def log_by_print(i, ts, y, pds):
    print("Interation: ", i)
    print("Theta: ", ts)
    print("y: ", y)
    print("pds: ",pds, "\n")

logger = to_csv("./adam-test.csv", ["t", "y", "t_d"])

def square(ts: List[T]) -> float: 
    # print("ts of square: ", ts)
    return (ts[0].inner - 1) ** 2

optimize(square, [[FloatAdam(100., 2, 0.8, 0.88)]], [], [[]], stop_if_small_y, logger)

([Float<inner: 0.4094398286438612>], 0.0009313902147921448)

In [6]:
# from math import copysign
# from gd.src import Arg


# class Int(Arg):
#     def __init__(self, inner: int) -> None:
#         super().__init__()
#         self.inner = inner

#     def succ(self) -> Arg:
#         return Int(self.inner + 1)
    
#     def update(self, cost: float, succ_cost: float) -> Arg:
#         diff = (succ_cost - cost)
#         return Int(self.inner - copysign(1, diff))
    
    
#     def __repr__(self) -> str:
#         return f"Int<inner: {self.inner}>"

In [7]:
# i = Int(1)
# print(i.succ())
# print(i.update(3., 2.))
# print(i.update(3., 2.))
# print(i.update(1., 2.))

In [8]:
# def square(args: Args) -> float:
#     return (args[0].inner - 7) ** 2 + (args[1].inner - 2) ** 2

# optimize(square, [Float(100.), Int(20)])

In [9]:
# a = [Float(100.), Int(20)]
# succ_i(1, a)

In [10]:
# a